In [ ]:
# The is only relevant to running mpi4py in a Jupyter notebook.
import ipyparallel
cluster=ipyparallel.Client(profile='mpi_tutorial')
print("IDs:",cluster.ids)

In [ ]:
%%px
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print ("I'm rank %d of %d on %s" %(rank,size,MPI.Get_processor_name()))

In [3]:
%%px

import numpy as np
import matplotlib.pyplot as plt 
import progressbar
from scipy import linalg as la
from scipy.sparse import identity
from scipy.sparse import rand
from scipy.sparse import diags
from scipy.sparse import triu

def gen_model(dim):
    ## dim is dimension value
    dim_o=dim
    A1 = -rand(dim,dim,density=0.75).toarray()/5
    A2 = triu(A1, k=1).toarray()/(10)
    A = diags(np.random.normal(-0.5,0,dim),0).toarray()/50 + A2 - A2.T
    ## we denote R1^{1/2},R2^{1/2} as R1,R2 repectively for convenience
    ## Non-Identity covariance matrix
    R1=(identity(dim).toarray() + np.tri(dim,dim,1) - np.tri(dim,dim,-2))/2
    R2=(identity(dim_o).toarray() + np.tri(dim_o,dim_o,1) - np.tri(dim_o,dim_o,-2))/2

    H=rand(dim_o,dim,density=0.75).toarray()/20
    m0=np.zeros(dim)+6
    C0=identity(dim).toarray()
    ## Collection of input 
    collection_input = [dim,dim_o,A,R1,R2,H,m0,C0]
    return collection_input




def gen_data(T,l,collection_input):
    
    [dim,dim_o,A,R1,R2,H,m0,C0]=collection_input
    J=T*(2**l)
    I=identity(dim).toarray()
    tau=2**(-l)
    L=la.expm(A*tau)
    ## We are going to need W to be symmetric! 
    W=(R1@R1)@(la.inv(A+A.T)@(L@(L.T)-I))
    C=tau*H
    V=(R2@R2)*tau

    v=np.zeros((J+1,dim,1))
    z=np.zeros((J+1,dim_o,1))
    v[0]=np.random.multivariate_normal(m0,C0,(1)).T
    z[0]=np.zeros((dim_o,1))


    for j in range(J):
        ## truth
        v[j+1] = L@v[j] + np.random.multivariate_normal(np.zeros(dim),W,(1)).T
        ## observation
        z[j+1] = z[j] + C@v[j+1] + np.random.multivariate_normal(np.zeros(dim_o),V,(1)).T
        
    return([z,v])

def cut(T,lmax,l,v):
    ind = np.arange(T*2**l+1)
    rtau = 2**(lmax-l)
    w = v[ind*rtau]
    return(w)

def EnKBF(T,l,lmax,z,N,collection_input):
    
    [dim,dim_o,A,R1,R2,H,m0,C0]=collection_input
    J=T*(2**l)
    I=identity(dim).toarray()
    I_o=identity(dim_o).toarray()
    dt=2**(-l)
    
    m=np.zeros((J+1,dim,1))
    c=np.zeros((J+1,dim,dim))
    z=cut(T,lmax,l,z)
    
    ## This gives a dim*N matrix
    x = np.random.multivariate_normal(m0,C0,N).T
    ## A dim*1 vector
    m[0]=(np.mean(x, axis=1)).reshape(dim,1)
    ## dim*dim matrix
    c[0]=((x-m[0])@((x-m[0]).T)) /(N-1)
    for j in range(J):
        dw = np.random.multivariate_normal(np.zeros(dim),dt*I,N).T
        dv = np.random.multivariate_normal(np.zeros(dim_o),dt*I_o,N).T
        ## A@x:dim*N R1@dw:dim*N c[j]@(H.T):dim*dim_o z[j+1]-z[j]:dim_o*1 H@x*dt:dim_o*N R2*dv:dim_o*N 
        ## x-m[j]:dim*N c[j]:dim*dim
        
        step1=(((x-m[j]).T)@(H.T))
        step2=step1@(la.inv(R2)@la.inv(R2))
        step3=step2@( (z[j+1]-z[j]) - (H@x*dt + R2@dv) )
        step4=(x-m[j])@step3 /(N-1)
        
        x = x + A@x*dt + R1@dw + step4
        m[j+1] = (np.mean(x, axis=1)).reshape(dim,1)

    return([m,c])

def CEnKBF(T,l,lmax,z,N,collection_input):
    
    [dim,dim_o,A,R1,R2,H,m0,C0]=collection_input
    J=T*(2**(l-1))
    I=identity(dim).toarray()
    I1=identity(dim_o).toarray()
    dt=2**(-l)
    dt1=2**(-l+1)
    
    m=np.zeros((J*2+1,dim,1))
    m1=np.zeros((J+1,dim,1))
    c=np.zeros((J*2+1,dim,dim))
    c1=np.zeros((J+1,dim,dim))
    z1=cut(T,lmax,l-1,z)
    z=cut(T,lmax,l,z)
    
    ## This gives a dim*N matrix
    x = np.random.multivariate_normal(m0,C0,N).T
    x1 = x
    ## A dim*1 vector
    m[0]=(np.mean(x, axis=1)).reshape(dim,1)
    m1[0]=m[0]
    ## dim*dim matrix
    c[0]=((x-m[0])@((x-m[0]).T)) /(N-1)
    c1[0]=c[0]
    
    dw=np.zeros((2,dim,N))
    dv=np.zeros((2,dim_o,N))
    for j in range(J):
        for s in range(2):
            dw[s] = np.random.multivariate_normal(np.zeros(dim),dt*I,N).T
            dv[s] = np.random.multivariate_normal(np.zeros(dim_o),dt*I1,N).T
            ## A@x:dim*N R1@dw:dim*N c[j]@(H.T):dim*dim_o z[j+1]-z[j]:dim_o*1 H@x*dt:dim_o*N R2*dv:dim_o*N 
            ## x-m[j]:dim*N c[j]:dim*dim

            step1=(((x-m[2*j+s]).T)@(H.T))
            step2=step1@(la.inv(R2)@la.inv(R2))
            step3=step2@( (z[2*j+s+1]-z[2*j+s]) - (H@x*dt + R2@dv[s]) )
            step4=(x-m[2*j+s])@step3 /(N-1)

            x = x + A@x*dt + R1@dw[s] + step4
            m[2*j+s+1] = (np.mean(x, axis=1)).reshape(dim,1)
        
        step1=(((x1-m1[j]).T)@(H.T))
        step2=step1@(la.inv(R2)@la.inv(R2))
        step3=step2@( (z1[j+1]-z1[j]) - (H@x1*dt1 + R2@(dv[0]+dv[1])) )
        step4=(x1-m1[j])@step3 /(N-1)
        
        x1 = x1 + A@x1*dt1 + R1@(dw[0]+dw[1]) + step4
        m1[j+1] = (np.mean(x1, axis=1)).reshape(dim,1)
        
    return([m,m1])
  
def coef(x, y): 
    # number of observations/points 
    n = np.size(x) 
  
    # mean of x and y vector 
    m_x, m_y = np.mean(x), np.mean(y) 
  
    # calculating cross-deviation and deviation about x 
    SS_xy = np.sum(y*x) - n*m_y*m_x 
    SS_xx = np.sum(x*x) - n*m_x*m_x 
  
    # calculating regression coefficients 
    b_1 = SS_xy / SS_xx 
    b_0 = m_y - b_1*m_x 
  
    return np.asarray((b_0, b_1)) 

### Function to check the convergence rate of $\mathbb{E}[\|(\eta_{t}^{l}-\eta_{t}^{l-1})^{N_l}(e) - (\eta_{t}^{l}-\eta_{t}^{l-1})(e)\|^2] \approx C_{3} \frac{\Delta_{l}}{N_{l}}$

In [4]:
%%px

def fit_var_cenkbf(seed_val):
    np.random.seed(seed_val)
    dim = 10
    lmax=9
    N = 50
    collection_input = gen_model(dim)
    z = gen_data(T,lmax,collection_input)[0]
    delta_seq = np.zeros(10)
    for l in range(10):
        delta_seq[l] = 2**(-l)
    
    Rep=100
    evar=np.zeros(10)
    for l in range(4,8):
        est = np.zeros(Rep)
        for rep in range(Rep):
            with np.errstate(divide='ignore'):
                [m,m1]=CEnKBF(T,l,lmax,z,N,collection_input)
                est[rep] = np.mean(m[-1,:]) - np.mean(m1[-1,:])
        evar[l] = np.var(est)
        
    x = np.log10(delta_seq[4:8])
    y = np.log10(evar[4:8])
    b=coef(x,y)
    return b

##### MPI4py implementation: 

##### 1. Each iteration use different iter_num

In [5]:
%%px
iter_num = 6

rank = comm.Get_rank()
size = comm.Get_size()

## Every iteration should have different initial_seed values
initial_seed = iter_num*(size)

seed_val_rankwise = initial_seed + rank

###### 2. Computation Cell: mute warnings and count the time

In [ ]:
%%px
import warnings
warnings.filterwarnings("ignore")
import time
import datetime

T = 100
stime = time.time()
rate_reptition = np.zeros(2)
rate_reptition = fit_var_cenkbf(seed_val_rankwise)
etime = time.time()
time_len = str(datetime.timedelta(seconds=etime-stime))
print("Time cost is:",time_len)

##### Result compilation & Filter out NaN values

In [ ]:
%%px

result = np.zeros((size,2))
comm.Gather(rate_reptition,result,root=0)
if rank == 0 :
    x = np.asarray(result)
    len_x = int(x[~np.isnan(x)].shape[0]/2)
    result_array = x[~np.isnan(x)].reshape((len_x,2))
    print("From seed val",initial_seed,'to',initial_seed+size,'result coefficients arrays are:')
    print(repr(result_array))

##### For High dimesional Implementation, we assume all the matrices in our model are of block diagonal structure, with each block of shape $10 \times 10$. Then the high dimensional implementaion of EnKBF/MLEnKBF can be seperated into block-wise implementation.

##### By Compiling the rate fitted through massive repetitions of low dim models, we can them compute the rate for high dimensional models.